<a href="https://colab.research.google.com/github/asheta66/Generative-AI-2/blob/main/B_Class_Autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import necessary libraries
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import seaborn as sns

# Set the directory for the data stored on Google Drive
data_dir = '/content/drive/My Drive/BrainTumor/56x56'

# Define image size and batch size for data generators
img_size = (128, 128)  # Adjust according to your data
batch_size = 32

# Set up data generators for training and validation with 20% validation split
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_generator = datagen.flow_from_directory(data_dir, target_size=img_size, batch_size=batch_size, class_mode='input', subset='training')
val_generator = datagen.flow_from_directory(data_dir, target_size=img_size, batch_size=batch_size, class_mode='input', subset='validation')

# Determine the number of classes from the data generator
num_classes = len(train_generator.class_indices)

# Define the Autoencoder model structure
input_img = Input(shape=(img_size[0], img_size[1], 3))

# Encoder part
x = Flatten()(input_img)
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
encoded = Dense(64, activation='relu')(x)

# Decoder part
x = Dense(128, activation='relu')(encoded)
x = Dense(256, activation='relu')(x)
x = Dense(img_size[0] * img_size[1] * 3, activation='sigmoid')(x)
decoded = Reshape((img_size[0], img_size[1], 3))(x)

# Compile the Autoencoder model
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='mse')

# Train the Autoencoder model
history = autoencoder.fit(train_generator, epochs=10, validation_data=val_generator)

# Extract the encoder part of the Autoencoder for feature extraction
encoder = Model(inputs=input_img, outputs=encoded)

# Define the classification model using the encoded features
encoded_input = Input(shape=(64,))
x = Dense(64, activation='relu')(encoded_input)
x = Dense(num_classes, activation='softmax')(x)
classifier = Model(inputs=encoded_input, outputs=x)

# Compile the classifier model with categorical crossentropy loss
classifier.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
classifier.summary()

# Generate encoded features for training and validation data
train_features = encoder.predict(train_generator)
val_features = encoder.predict(val_generator)

# Get the corresponding labels and convert them to one-hot encoding
train_labels = train_generator.classes
val_labels = val_generator.classes

train_labels = to_categorical(train_labels, num_classes=num_classes)
val_labels = to_categorical(val_labels, num_classes=num_classes)

# Train the classifier model using the encoded features
history_classifier = classifier.fit(train_features, train_labels, epochs=50, validation_data=(val_features, val_labels))

# Evaluate the classifier using the training data
train_predictions = np.argmax(classifier.predict(train_features), axis=1)
train_labels_original = np.argmax(train_labels, axis=1)

# Evaluate the classifier using the validation data
val_predictions = np.argmax(classifier.predict(val_features), axis=1)
val_labels_original = np.argmax(val_labels, axis=1)

Mounted at /content/drive
Found 2452 images belonging to 3 classes.
Found 612 images belonging to 3 classes.
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


66/77 ━━━━━━━━━━━━━━━━━━━━ 1:52 10s/step - loss: 0.0231

KeyboardInterrupt: 

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Define figure size for confusion matrix plots
fig_size = (7, 4)  # Adjust the size as needed

# Calculate and display the confusion matrix for training data
conf_matrix_train = confusion_matrix(np.argmax(train_labels, axis=1), np.argmax(train_predictions, axis=1))

# Create a figure with the specified size
plt.figure(figsize=fig_size)
sns.heatmap(conf_matrix_train, annot=True, fmt="d", cmap='Blues', xticklabels=train_generator.class_indices.keys(), yticklabels=train_generator.class_indices.keys())
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Training Confusion Matrix')
plt.show()

# Calculate and display the confusion matrix for validation data
conf_matrix_val = confusion_matrix(np.argmax(val_labels, axis=1), np.argmax(val_predictions, axis=1))

# Create a figure with the specified size
plt.figure(figsize=fig_size)
sns.heatmap(conf_matrix_val, annot=True, fmt="d", cmap='Blues', xticklabels=val_generator.class_indices.keys(), yticklabels=val_generator.class_indices.keys())
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Validation Confusion Matrix')

# Adjust layout to prevent overlap
plt.tight_layout()

# Save the figure to a file in Colab's local filesystem
plt.savefig('/content/Confusion Matrix.png')

# Display the plots
plt.show()


# Print the classification report for training data
print("Training Classification Report:")
print(classification_report(train_labels_original, train_predictions, target_names=train_generator.class_indices.keys()))

# Print the classification report for validation data
print("Validation Classification Report:")
print(classification_report(val_labels_original, val_predictions, target_names=val_generator.class_indices.keys()))

# Calculate accuracy, precision, recall, and F1-score for training data
train_accuracy = accuracy_score(train_labels_original, train_predictions)
train_precision = precision_score(train_labels_original, train_predictions, average='weighted')
train_recall = recall_score(train_labels_original, train_predictions, average='weighted')
train_f1 = f1_score(train_labels_original, train_predictions, average='weighted')

print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Training Precision: {train_precision:.4f}")
print(f"Training Recall: {train_recall:.4f}")
print(f"Training F1 Score: {train_f1:.4f}")

# Calculate accuracy, precision, recall, and F1-score for validation data
val_accuracy = accuracy_score(val_labels_original, val_predictions)
val_precision = precision_score(val_labels_original, val_predictions, average='weighted')
val_recall = recall_score(val_labels_original, val_predictions, average='weighted')
val_f1 = f1_score(val_labels_original, val_predictions, average='weighted')

print(f"Validation Accuracy: {val_accuracy:.4f}")
print(f"Validation Precision: {val_precision:.4f}")
print(f"Validation Recall: {val_recall:.4f}")
print(f"Validation F1 Score: {val_f1:.4f}")

import matplotlib.pyplot as plt

# Define the figure and subplots
plt.figure(figsize=(7, 4))

# Plot training and validation loss curves for the Autoencoder
plt.subplot(2, 1, 1)  # 2 rows, 1 column, 1st subplot
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Autoencoder Loss')
plt.legend()

# Plot training and validation accuracy curves for the classifier
plt.subplot(2, 1, 2)  # 2 rows, 1 column, 2nd subplot
plt.plot(history_classifier.history['accuracy'], label='train_acc')
plt.plot(history_classifier.history['val_accuracy'], label='val_acc')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Classifier Accuracy')
plt.legend()

# Adjust layout to prevent overlap
plt.tight_layout()

# Save the figure to a file in Colab's local filesystem
plt.savefig('/content/autoencoder_classifier_curves.png')

# Display the plots
plt.show()

# Print the summary of the Autoencoder model
autoencoder.summary()
